# **Deep Learning**

The term **deep learning** refers to a subset of machine learning techniques that utilize artificial **neural networks** with multiple layers — hence "deep" — to model and understand complex patterns in data.

<div align="center">
    <img src="assets/ai-ml-dl.svg" height="300" />
    <p><em>Deep learning (DL) is a subset of machine learning (ML), which is a subset of artificial intelligence (AI).</em></p>
</div>

Just as classic machine learning, deep learning algorithms can be classified into three main categories:
- **Supervised learning**: the model is trained on a labeled dataset, meaning that each input data point is paired with the correct output. The model learns to map inputs to outputs by minimizing the error between its predictions and the actual labels. Common applications include image classification, speech recognition, and natural language processing.
- **Unsupervised learning**: it involves training models on unlabeled data. The goal is to discover hidden patterns or structures within the data without predefined labels. Techniques such as autoencoders and generative adversarial networks (GANs) are often used for tasks like clustering, dimensionality reduction, and data generation.
- **Reinforcement learning**: models learn to make decisions by interacting with an environment. The model, often referred to as an agent, receives feedback in the form of rewards or penalties based on its actions. Deep reinforcement learning has been successfully applied in areas such as game playing (e.g., AlphaGo), robotics, and autonomous systems.

In particular, we'll motivate deep learning from an **image classification** perspective, which is a classic supervised learning task where the goal is to assign a label to an input image based on its content.

## **Handwriting Recognition**

Suppose we've been tasked with building a function that takes a $28 \times 28 $ grayscale image of a handwritten digit ($0-9$) as input and outputs the corresponding digit label.

<div align="center">
    <img src="assets/handwritten-digits.png" height="300" />
    <p><em>Handwritten digits</em></p>
</div>

Let $x \in \mathbb{R}^{28 \times 28}$ be the input image, and $y \in \mathbb{Z}_{10}$ be the output label, where $\mathbb{Z}_{10} = \{0, 1, \ldots, 9\}$. Then, we want to write a function $f$ such that:

$$
f(x) = y.
$$

Similarly, from a programming perspective, we want to implement a function that takes an array as input and returns an integer as output:

```python
def f(x: np.ndarray) -> int:
    digit = ...
    return digit
```

As human programmers, we can easily recognize handwritten digits by looking at the images. However, developing an algorithm that can accurately perform this task is no trivial endeavor.

Without loss of generality, we consider the universe of all functions $f$ that can map grayscale images to digits, this is, matrices to scalars:

$$
f: \mathbb{R}^{28 \times 28} \to \mathbb{Z}_{10},
$$

so that the function $f^\ast$ that we are looking for — this is, the one that correctly maps images to the digits they display — is guaranteed to belong to this set.

In order to make searching for $f^\ast$ tractable, we reframe our problem as a **deep supervised learning** task.

## **Neural Networks**

Since the space of all functions that map images to digits is uncountably infinite, the **deep supervised learning** paradigm reduces the search space of all possible functions $f: \mathbb{R}^{28 \times 28} \to \mathbb{Z}_{10}$ to a smaller subset of this space specified by a given **neural network** architecture. Then, $f$ defines the family of functions parameterized by a set of weights $\theta$, according to the chosen architecture:

$$
f(x; \theta) = y.
$$

<div align="center">
    <img src="assets/nn.png" height="300" />
    <p><em>Feedforward Neural Network (FNN)</em></p>
</div>

Now that our neural net depends both on the input $x$ and the parameters $\theta$, we need to find a way of finding the parameters $\theta^\ast$ such that $f(x; \theta^\ast)$ approximates $f^\ast(x)$ as closely as possible for all possible inputs $x$, this is, all possible $28 \times 28$ images of handwritten digits.

### **Dataset**

First, since optimizing $\theta$ over all possible inputs is infeasible, we  rely on a finite **dataset** of $N$ labeled examples:

$$
\mathcal{D} = \{(x_i, y_i)\}_{i=1}^N,
$$

where each $x_i$ is a $28 \times 28$ grayscale image of a handwritten digit, and $y_i$ is the corresponding label.

For our **handwriting recognition** task, we use the public [**MNIST dataset**](http://yann.lecun.com/exdb/mnist/).

<div align="center">
    <img src="assets/mnist-dataset.png" height="300" />
    <p><em>MNIST dataset samples</em></p>
</div>

### **Loss Function**

Second, we need to define a measure of how well our function $f(x; \theta)$ approximates the true function $f^\ast(x)$ within our dataset $\mathcal{D}$. For that,
we use a **loss function** that quantifies the difference between the predicted output $f(x; \theta)$ and the true label $y$:

$$
\mathcal{L}(f(x; \theta), y).
$$

Since we model our handwriting recognition task as a **regression** problem — we want to predict a continuous output (the digit label) which we round to the nearest integer — we use the **mean squared error (MSE)** loss function:

$$
\mathcal{L}(f(x; \theta), y) = [f(x; \theta) - y]^2.
$$

<div align="center">
    <img src="assets/loss.png" height="300" />
    <p><em>Loss function illustration</em></p>
</div>

## **Optimization**

Having defined our dataset $\mathcal{D}$ and loss function $\mathcal{L}$, we can now formulate the optimization problem that will allow us to find the optimal parameters $\theta^\ast$ for our neural network:

\begin{align*}
\theta^\ast &= \arg \min_\theta \frac{1}{N} \sum_{i=1}^N \mathcal{L}(f(x_i; \theta), y_i),\\
&= \arg \min_\theta g(\theta).
\end{align*}

Therefore, given a fixed **network architecture** and **dataset**, our goal is to optimize the parameters $\theta$ so that the **loss function** is minimized.

### **Differentiation**

Then, finding the **neural network** $f(x; \theta^\ast)$ that best approximates the true function $f^\ast(x)$ is equivalent to minimizing $g(\theta)$ which, in mathematical terms, can be achieved by taking its **gradient** with respect to the parameters $\theta$ and setting it to zero:

$$
\nabla_\theta g(\theta) = 0.
$$

However, since neural networks are **non-linear** functions with potentially millions of parameters, solving this system of equations analytically is infeasible. We must therefore resort to **numerical optimization** techniques to find an approximate solution.

#### **Gradient Descent**

One of the most common numerical optimization techniques used in deep learning is **gradient descent**. This iterative algorithm updates the parameters $\theta$ in the direction of the negative gradient of the loss function, scaled by a learning rate $\alpha$:

$$
\theta_{t+1} = \theta_t - \alpha \nabla_\theta g(\theta_t).
$$

Hence, we now need an algorithm to efficiently compute the gradient of the loss function with respect to the parameters of the neural network at each iteration $t$.

<div align="center">
    <img src="assets/gradient-descent.png" height="300" />
    <p><em>Gradient descent</em></p>
</div>

#### **Backpropagation**

First, we need to understand why **symbolic** or **numerical differentation** methods are not suitable for computing the gradient of the loss function in deep learning:
- **Symbolic differentiation**: while it can provide exact gradients, it is impractical for deep neural networks because expressions become extremely large and complex, making symbolic manipulation inefficient and memory-intensive.
- **Numerical differentiation**: although it is straightforward to implement, it suffers from numerical instability, it is computationally expensive and can introduce significant approximation errors, especially in high-dimensional parameter spaces.

Instead, if we leverage the fact that neural networks are composed of a sequence of **layers**, which are differentiable operations, we can use the **backpropagation** algorithm to efficiently compute the gradient of the loss function with respect to the network parameters.

**Automatic differentiation** constructs a **computational graph** that allows computing exact gradients efficiently by systematically applying the **chain rule** of calculus to the operations performed in the neural network.

<div align="center">
    <img src="assets/computational-graph.png" height="300" />
    <p><em>Computational graph</em></p>
</div>

In particular, given that **neural nets** are compositions of elementary functions such as sums, products and non-linear activations, we can define the **forward** and **backward** passes of these operations and let the **backpropagation** algorithm compute the gradients automatically by composing the local gradients using the **chain rule** for any arbitrary neural network architecture.

In order to illustrate this point, in what follows we manually implement a simple **autodiff engine** with no dependencies and train a simple neural network for the **handwriting recognition** task. In this [notebook's GitHub repo](https://github.com/segusantos/deep-learning-notebook) you can find a more complete implementation with additional primitives and tests that validate its correctness against [PyTorch](https://pytorch.org/).

## **Autodiff Engine**

In [3]:
from __future__ import annotations

In [ ]:
class Scalar:
    def __init__(self,
                 data: int | float,
                 _backward: function = lambda: None,
                 _prev: list[Scalar] = []) -> None:
        self.data: float = float(data)
        self.grad: float = 0.0
        self._backward: function = _backward
        self._prev: list[Scalar] = _prev

    def backward(self) -> None:
        topological_order = []
        visited = set()
        def dfs(scalar: Scalar) -> None:
            if scalar not in visited:
                visited.add(scalar)
                for prev in scalar._prev:
                    dfs(prev)
                topological_order.append(scalar)
        dfs(self)
        self.grad = 1.0
        for scalar in reversed(topological_order):
            scalar._backward()

    def __repr__(self) -> str:
        return f"Scalar(data={self.data}, grad={self.grad})"